In [30]:
import pandas as pd
import os
import re
from pathlib import Path
import glob

#collect text file
# Use the for loop to iterate through the list you just created, and open the files
path = os.getcwd()
csv_files = glob.glob(os.path.join(path, "*.txt"))
# loop over the list of csv files
for f in csv_files:
    d = pd.read_csv(f, skiprows=17, sep='\t')
    d['Time'] = pd.to_datetime(d['Time'])
    d = d[1:]
    d = d[['Time', 'Stage']]

    d['Time'] = pd.to_datetime(d['Time'], format='%m/%d/%Y %H:%M:%S')
    ##round time
    d['Hour'] = d['Time'].dt.floor(freq='H')

    # Full StageList
    StageList = d[['Stage']]
    StageList = StageList.drop_duplicates()
    StageList['TimeInStage'] = 0
    StageList['key'] = 1

    # list out all of the dates used
    Hours = d[['Hour']]
    Hours = Hours.drop_duplicates()
    Hours['key'] = 1

    StageNotIdentified = pd.merge(Hours, StageList, on='key').drop("key", 1)
    StageNotIdentified = StageNotIdentified.groupby(['Hour', 'Stage']).sum('TimeInStage').reset_index()
    ##count as four seconds for all rows
    d['TimeInStage'] = 4

    d = d.groupby(['Hour', 'Stage']).sum('TimeInStage').reset_index()
    
    dMerge = d[['Hour', 'Stage']]
    StageNotIdentified = StageNotIdentified[['Hour', 'Stage']]

    outer_join = StageNotIdentified.merge(dMerge, how='outer', indicator=True)
    anti_join = outer_join[~(outer_join._merge == 'both')].drop('_merge', axis=1)
    anti_join['TimeInStage'] = 0

    d = pd.concat([d, anti_join])
    d = d.sort_values(['Hour', 'Stage'])
    d = d.groupby(['Stage'])['TimeInStage'].agg('sum')
    d=d.sort_values(ascending = False).head(1)
    d.to_csv('Output.csv')